In [ ]:
import os
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score , davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from sklearn.preprocessing import MinMaxScaler
from sklearn.semi_supervised import SelfTrainingClassifier
from scipy import stats

import time
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import roc_curve , auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve , roc_auc_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
%matplotlib inline

In [ ]:
labels = np.load('trainlabels.npy',allow_pickle=True)
labels_TEST = np.load('testlabels.npy',allow_pickle=True)

In [ ]:
len(labels)

In [ ]:
s_cleaned_array_list = np.load('traindata.npy',allow_pickle=True)
s_cleaned_array_list_TEST = np.load('testdata.npy',allow_pickle=True)

In [ ]:
s_cleaned_tot = list(s_cleaned_array_list)

s_cleaned_tot.extend(list(s_cleaned_array_list_TEST))

In [ ]:
len(s_cleaned_array_list)

In [ ]:
c_s = 0

tr = 2

for i in range(len(s_cleaned_array_list)):
    
    x = s_cleaned_array_list[i]
    
    dl_list_day = []
    
    e = s_cleaned_array_list[i][:,:24:2]
    
    for j in range(e.shape[0]):
        
        if np.sum(e[j]) <= tr :
            
            c_s += 1
            
            dl_list_day.append(j)
            
    x = np.delete(x,dl_list_day,axis=0)
    
    s_cleaned_array_list[i] = x

In [ ]:
c_s

In [ ]:
c_s = 0

tr = 2

for i in range(len(s_cleaned_array_list_TEST)):
    
    x = s_cleaned_array_list_TEST[i]
    
    dl_list_day = []
    
    e = s_cleaned_array_list_TEST[i][:,:24:2]
    
    for j in range(e.shape[0]):
        
        if np.sum(e[j]) <= tr :
            
            c_s += 1
            
            dl_list_day.append(j)
            
    x = np.delete(x,dl_list_day,axis=0)
    
    s_cleaned_array_list_TEST[i] = x

In [ ]:
c_s

In [ ]:
for i in range(len(s_cleaned_array_list_TEST)):
    
    print(len(s_cleaned_array_list_TEST[i]))

In [ ]:
energies = []
powers = []

for i in range(len(s_cleaned_tot)):

    e = s_cleaned_tot[i][:,:24:2].reshape(-1)
    p = s_cleaned_tot[i][:,1::2].reshape(-1)
    
    for j in range(e.shape[0]):
        
        energies.append(e[j])
        powers.append(p[j])

In [ ]:
power_med = np.percentile(powers,50)
energy_med = np.percentile(energies,50)

energy_q1 = np.percentile(energies,25)
power_q1 = np.percentile(powers,25)

energy_q3 = np.percentile(energies,75)
power_q3 = np.percentile(powers,75)

In [ ]:
energy_q3

In [ ]:
mean_s_e = []

mean_s_p = []

std_s_e = []

std_s_p = []


for i in range(len(s_cleaned_array_list)):
      
    e_s = s_cleaned_array_list[i][:,:24:2]
    p_s = s_cleaned_array_list[i][:,1:24:2]
    
    a = np.mean(e_s)
    c = np.mean(p_s)
       
    mean_s_e.append(np.sqrt(a))
    mean_s_p.append(np.sqrt(c))
    
    a = np.std(e_s)
    c = np.std(p_s)
    
    std_s_e.append(np.sqrt(a))
    std_s_p.append(np.sqrt(c))

In [ ]:
q1_e_ratio = []
q1_p_ratio = []

q2_e_ratio = []
q2_p_ratio = []

q3_e_ratio = []
q3_p_ratio = []

C_max_e = []
C_max_p = []

C_min_e = []
C_min_p = []

R_e_meanmax = []
R_p_meanmax = []

R_e_minmean = []
R_p_minmean = []


for i in range(len(s_cleaned_array_list)):
    
    c1 = 0
    c2 = 0
    
    c3 = 0
    c4 = 0
    
    c5 = 0
    c6 = 0
    
    e = s_cleaned_array_list[i][:,:24:2].reshape(-1)
    p = s_cleaned_array_list[i][:,1::2].reshape(-1)
    
    n = e.shape[0]
    
    for j in range(n):
        
        if e[j] >= energy_med:
            
            c1 += 1
        
        if p[j] >= power_med:
            
            c2 += 1

        if e[j] >= energy_q1:
            
            c3 += 1
        
        if p[j] >= power_q1:
            
            c4 += 1

        if e[j] >= energy_q3:
            
            c5 += 1
        
        if p[j] >= power_q3:
            
            c6 += 1
            
    q2_e_ratio.append(c1/n)
    q2_p_ratio.append(c2/n)

    q1_e_ratio.append(c3/n)
    q1_p_ratio.append(c4/n)
    
    q3_e_ratio.append(c5/n)
    q3_p_ratio.append(c6/n)
    
    C_max_e.append(np.max(e))
    C_max_p.append(np.max(p))

    C_min_e.append(np.log(np.min(e)+0.000001))
    C_min_p.append(np.log(np.min(p)+0.000001))
    
    if np.min(p) == 0 :
        
        print(i)
    
    R_e_meanmax.append(np.log(np.mean(e)/np.max(e)))
    R_p_meanmax.append(np.log(np.mean(p)/np.max(p)))

    R_e_minmean.append(np.sqrt(np.sqrt(np.min(e)/np.mean(e))))
    R_p_minmean.append(np.sqrt(np.sqrt(np.min(p)/np.mean(p))))

In [ ]:
X = np.transpose(np.vstack((q1_e_ratio,q1_p_ratio,q2_e_ratio,q2_p_ratio,q3_e_ratio,q3_p_ratio,
                            C_max_e,C_max_p,C_min_e,C_min_p,R_e_meanmax,R_p_meanmax,R_e_minmean,R_p_minmean
                            ,mean_s_e,mean_s_p,std_s_e,std_s_p)))

In [ ]:
X.shape

In [ ]:
X

In [ ]:
means_e = []

means_p = []

stds_e = []

stds_p = []


for i in range(len(s_cleaned_array_list)):
      
    e_s = s_cleaned_array_list[i][:,:24:2]
    p_s = s_cleaned_array_list[i][:,1:24:2]
    
       
    means_e.append(np.sqrt(np.mean(e_s,axis=0)))
    means_p.append(np.sqrt(np.mean(p_s,axis=0)))
    
    stds_e.append(np.sqrt(np.std(e_s,axis=0)))
    stds_p.append(np.sqrt(np.std(p_s,axis=0)))

In [ ]:
X = np.hstack((X,means_e,means_p,stds_e,stds_p))

In [ ]:
X.shape

In [ ]:
mean_s_e = []

mean_s_p = []

std_s_e = []

std_s_p = []


for i in range(len(s_cleaned_array_list_TEST)):
      
    e_s = s_cleaned_array_list_TEST[i][:,:24:2]
    p_s = s_cleaned_array_list_TEST[i][:,1:24:2]
    
    a = np.mean(e_s)
    c = np.mean(p_s)
       
    mean_s_e.append(a)
    mean_s_p.append(c)
    
    a = np.std(e_s)
    c = np.std(p_s)
    
    std_s_e.append(a)
    std_s_p.append(c)

In [ ]:
q1_e_ratio = []
q1_p_ratio = []

q2_e_ratio = []
q2_p_ratio = []

q3_e_ratio = []
q3_p_ratio = []

C_max_e = []
C_max_p = []

C_min_e = []
C_min_p = []

R_e_meanmax = []
R_p_meanmax = []

R_e_minmean = []
R_p_minmean = []


for i in range(len(s_cleaned_array_list_TEST)):
    
    c1 = 0
    c2 = 0
    
    c3 = 0
    c4 = 0
    
    c5 = 0
    c6 = 0
    
    e = s_cleaned_array_list_TEST[i][:,:24:2].reshape(-1)
    p = s_cleaned_array_list_TEST[i][:,1::2].reshape(-1)
    
    n = e.shape[0]
    
    for j in range(n):
        
        if e[j] >= energy_med:
            
            c1 += 1
        
        if p[j] >= power_med:
            
            c2 += 1

        if e[j] >= energy_q1:
            
            c3 += 1
        
        if p[j] >= power_q1:
            
            c4 += 1

        if e[j] >= energy_q3:
            
            c5 += 1
        
        if p[j] >= power_q3:
            
            c6 += 1
            
    q2_e_ratio.append(c1/n)
    q2_p_ratio.append(c2/n)

    q1_e_ratio.append(c3/n)
    q1_p_ratio.append(c4/n)
    
    q3_e_ratio.append(c5/n)
    q3_p_ratio.append(c6/n)
    
    C_max_e.append(np.max(e))
    C_max_p.append(np.max(p))

    C_min_e.append(np.log(np.min(e)+0.000001))
    C_min_p.append(np.log(np.min(p)+0.000001))
    
    R_e_meanmax.append(np.log(np.mean(e)/np.max(e)))
    R_p_meanmax.append(np.log(np.mean(p)/np.max(p)))

    R_e_minmean.append(np.sqrt(np.sqrt(np.min(e)/np.mean(e))))
    R_p_minmean.append(np.sqrt(np.sqrt(np.min(p)/np.mean(p))))

In [ ]:
x_test = np.transpose(np.vstack((q1_e_ratio,q1_p_ratio,q2_e_ratio,q2_p_ratio,q3_e_ratio,q3_p_ratio,
                            C_max_e,C_max_p,C_min_e,C_min_p,R_e_meanmax,R_p_meanmax,R_e_minmean,R_p_minmean
                            ,mean_s_e,mean_s_p,std_s_e,std_s_p)))

In [ ]:
x_test.shape

In [ ]:
means_e = []

means_p = []

stds_e = []

stds_p = []


for i in range(len(s_cleaned_array_list_TEST)):
      
    e_s = s_cleaned_array_list_TEST[i][:,:24:2]
    p_s = s_cleaned_array_list_TEST[i][:,1:24:2]
    
       
    means_e.append(np.sqrt(np.mean(e_s,axis=0)))
    means_p.append(np.sqrt(np.mean(p_s,axis=0)))
    
    stds_e.append(np.sqrt(np.std(e_s,axis=0)))
    stds_p.append(np.sqrt(np.std(p_s,axis=0)))

In [ ]:
x_test = np.hstack((x_test,means_e,means_p,stds_e,stds_p))

In [ ]:
x_test.shape

In [ ]:
x_test[0]

In [ ]:
y_train = labels
y_test = labels_TEST

In [ ]:
x_train = X

In [ ]:
c_list = [0.001 , 0.01 , 0.1 , 1 , 10 , 100 , 100]

In [ ]:
train_scores = []
test_scores = []

for i in c_list:

    RF_clf =  SVC(C =i  , class_weight = 'balanced', tol = 1e-6 , probability = True  )

    test_list = []
    train_list = []

    for j in range(1):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
values = [i for i in range(1, 8)]
# plot of train and test scores vs tree depth
plt.plot(values, train_scores, '-o', label='Train')
plt.plot(values, test_scores, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
train_scores = []
test_scores = []

for i in c_list:

    RF_clf =  SVC(C =i, tol = 1e-6 , probability = True  )

    test_list = []
    train_list = []

    for j in range(1):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
values = [i for i in range(1, 8)]
# plot of train and test scores vs tree depth
plt.plot(values, train_scores, '-o', label='Train')
plt.plot(values, test_scores, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
test_scores

In [ ]:
X_test = x_test

In [ ]:
X_tr_train = X[:11]
X_tr_test = X_test[:11]

In [ ]:
X_tr_train.shape

In [ ]:
X.shape

In [ ]:
y_train[:11]

In [ ]:
y_test[:11]

In [ ]:
X_tr_test

In [ ]:
X_tr_train

In [ ]:
x_test = np.vstack((X_tr_train,x_test[11:]))

x_train = np.vstack((X_tr_test,x_train[11:]))

In [ ]:
x_test

In [ ]:
x_train

In [ ]:
X_tr_train

In [ ]:
train_scores = []
test_scores = []

for i in c_list:

    RF_clf =  SVC(C =i  , class_weight = 'balanced', tol = 1e-6 , probability = True  )

    test_list = []
    train_list = []

    for j in range(1):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
values = [i for i in range(1, 8)]
# plot of train and test scores vs tree depth
plt.plot(values, train_scores, '-o', label='Train')
plt.plot(values, test_scores, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
c_list

In [ ]:
test_scores

In [ ]:
test_scores

In [ ]:
train_scores = []
test_scores = []

for i in c_list:

    RF_clf =  SVC(C =i, tol = 1e-6 , probability = True  )

    test_list = []
    train_list = []

    for j in range(1):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
values = [i for i in range(1, 8)]
# plot of train and test scores vs tree depth
plt.plot(values, train_scores, '-o', label='Train')
plt.plot(values, test_scores, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
t1 =time.perf_counter()

GSVM_clf = SVC(C = 10, tol = 1e-6 , probability = True)
GSVM_clf.fit(x_train,y_train)

t2 = time.perf_counter()

t_LR = t2-t1

print(t_LR)

In [ ]:
y_pred=GSVM_clf.predict(x_test)

ts_GSVM = GSVM_clf.score(x_test,y_test)
tr_GSVM = GSVM_clf.score(x_train,y_train)

print("%s: test Accuracy = %.4f%% - train Accuracy = %.4f%%" % ("Gaussian SVM", ts_GSVM, tr_GSVM ))

cm_GSVM=confusion_matrix(y_test,y_pred)
print(cm_GSVM)
print(classification_report(y_test,y_pred, digits = 4))

In [ ]:
yproba = GSVM_clf.predict_proba(x_test)[:,1]

In [ ]:
roc_auc_score(y_test, yproba)

In [ ]:
t1 =time.perf_counter()

GSVM_clf = SVC(C = 10 , class_weight = 'balanced', tol = 1e-6 , probability = True)
GSVM_clf.fit(x_train,y_train)

t2 = time.perf_counter()

t_LR = t2-t1

print(t_LR)

In [ ]:
y_pred=GSVM_clf.predict(x_test)

ts_GSVM = GSVM_clf.score(x_test,y_test)
tr_GSVM = GSVM_clf.score(x_train,y_train)

print("%s: test Accuracy = %.4f%% - train Accuracy = %.4f%%" % ("Gaussian SVM", ts_GSVM, tr_GSVM ))

cm_GSVM=confusion_matrix(y_test,y_pred)
print(cm_GSVM)
print(classification_report(y_test,y_pred, digits = 4))

In [ ]:
yproba = GSVM_clf.predict_proba(x_test)[::,1]

In [ ]:
roc_auc_score(y_test, yproba)

In [ ]:
train_scores = []
test_scores = []

for i in range(1,51):

    RF_clf = KNeighborsClassifier(n_neighbors=i)

    test_list = []
    train_list = []

    for j in range(1):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
values = [i for i in range(1, 51)]
# plot of train and test scores vs tree depth
plt.plot(values, train_scores, '-o', label='Train')
plt.plot(values, test_scores, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
t1 =time.perf_counter()

GNB_clf = GaussianNB()
GNB_clf.fit(x_train,y_train)

t2 = time.perf_counter()

t_LR = t2-t1

print(t_LR)

In [ ]:
y_pred=GNB_clf.predict(x_test)

ts_GNB = GNB_clf.score(x_test,y_test)
tr_GNB = GNB_clf.score(x_train,y_train)

print("%s: test Accuracy = %.4f%% - train Accuracy = %.4f%%" % ("naive bayes", ts_GNB, tr_GNB ))

cm_GNB=confusion_matrix(y_test,y_pred)
print(cm_GNB)
print(classification_report(y_test,y_pred, digits = 4))

In [ ]:
train_scores = []
test_scores = []

for i in range(1,11):

    RF_clf = RandomForestClassifier(class_weight='balanced',max_depth = i)

    test_list = []
    train_list = []

    for j in range(100):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
# plot of train and test scores vs tree depth
plt.plot(values[:10], train_scores[:10], '-o', label='Train')
plt.plot(values[:10], test_scores[:10], '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
e_s.shape

In [ ]:
labels = list(labels_gas)

labels.extend(list(labels_water))

labels.extend(list(labels_centeral))

labels.extend(list(labels_nil))

In [ ]:
s_cleaned_array_list_nil = np.load('niloofar_array_list.npy',allow_pickle=True)

In [ ]:
s_cleaned_array_list = list(s_cleaned_array_list_gas)

s_cleaned_array_list.extend(list(s_cleaned_array_list_water))

s_cleaned_array_list.extend(list(s_cleaned_array_list_centeral))

s_cleaned_array_list.extend(list(s_cleaned_array_list_nil))

In [ ]:
len(s_cleaned_array_list)

In [ ]:
c_s = 0

tr = 2

for i in range(len(s_cleaned_array_list)):
    
    x = s_cleaned_array_list[i]
    
    dl_list_day = []
    
    e = s_cleaned_array_list[i][:,:24:2]
    
    for j in range(e.shape[0]):
        
        if np.sum(e[j]) <= tr :
            
            c_s += 1
            
            dl_list_day.append(j)
            
    x = np.delete(x,dl_list_day,axis=0)
    
    s_cleaned_array_list[i] = x

In [ ]:
c_s

In [ ]:
dl_list = []

for i in range(len(s_cleaned_array_list)):
    
    if len(s_cleaned_array_list[i]) <= 7:
        
        dl_list.append(i)

In [ ]:
len(dl_list)

In [ ]:
s_cleaned_array_list = np.delete(s_cleaned_array_list,dl_list,axis = 0)

In [ ]:
labels = np.delete(labels,dl_list,axis = 0)

In [ ]:
len(labels),len(s_cleaned_array_list)

In [ ]:
np.unique(labels,return_counts=True)

In [ ]:
labels_2c = []

for i in range(len(labels)):
    
    if labels[i] != 2:
            
        labels_2c.append(0)
            
    else:
            
        labels_2c.append(1)

In [ ]:
len(labels_2c)

In [ ]:
mean_s_e = []

mean_s_p = []

mean_a_e = []

mean_a_p = []

std_s_e = []

std_s_p = []

std_a_e = []

std_a_p = []

for i in range(len(s_cleaned_array_list)):
      
    e_s = s_cleaned_array_list[i][:,:24:2]
    p_s = s_cleaned_array_list[i][:,1:24:2]

        
    
    a = np.mean(e_s)
    c = np.mean(p_s)
       
    mean_s_e.append(a)
    mean_s_p.append(c)

    
    a = np.std(e_s)
    c = np.std(p_s)

    std_s_e.append(a)
    std_s_p.append(c)


In [ ]:
X_old = np.transpose(np.vstack((mean_s_e,mean_s_p,std_s_e,std_s_p)))

In [ ]:
X_old.shape

In [ ]:
X = X_old
y = labels_2c

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 57 , random_state=127)

In [ ]:
train_scores = []
test_scores = []

for i in range(1,351):

    RF_clf =  SVC(C =i  , class_weight = 'balanced', tol = 1e-6 , probability = True )

    test_list = []
    train_list = []

    for j in range(1):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
values = [i for i in range(1, 351)]
# plot of train and test scores vs tree depth
plt.plot(values, train_scores, '-o', label='Train')
plt.plot(values, test_scores, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:

plt.plot(values[100:150], train_scores[100:150], '-o', label='Train')
plt.plot(values[100:150], test_scores[100:150], '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
for i in range(len(s_cleaned_array_list)):
    
    x_train, x_test, y_train, y_test = (np.vstack((X[:i],X[i+1:])) , X[i,:] , np.vstack((np.array(y[:i]).reshape((-1,1)),np.array(y[i+1:]).reshape((-1,1)))), y[i] )
    
    
    RF_clf =  SVC(C = 2  , class_weight = 'balanced', tol = 1e-6 , probability = True )


    RF_clf.fit(x_train,y_train)

    y_pred=RF_clf.predict(x_test.reshape(1, -1))
    
    prob = RF_clf.predict_proba(x_test.reshape(1, -1))[0]
    
    if y_test != y_pred[0]:
        
        print(i,y_test,prob[0],y_pred)

In [ ]:
for i in range(len(s_cleaned_array_list)):
    
    x_train, x_test, y_train, y_test = (np.vstack((X[:i],X[i+1:])) , X[i,:] , np.vstack((np.array(y[:i]).reshape((-1,1)),np.array(y[i+1:]).reshape((-1,1)))), y[i] )
    
    
    RF_clf =  SVC(C = 120  , class_weight = 'balanced', tol = 1e-6 , probability = True )


    RF_clf.fit(x_train,y_train)

    y_pred=RF_clf.predict(x_test.reshape(1, -1))
    
    prob = RF_clf.predict_proba(x_test.reshape(1, -1))[0]
    
    if y_test != y_pred[0]:
        
        print(i,y_test,prob,y_pred)

In [ ]:
for i in range(len(s_cleaned_array_list)):

    
    f = plt.figure(figsize=(15,5))
    ax = f.add_subplot(121)
    ax2 = f.add_subplot(122)
    plt.title(str(i))
    ax.plot(s_cleaned_array_list[i][:,:24:2].reshape(-1))
    plt.show()

In [ ]:
dl_list_ml = [99,165,180,181,182,265]

In [ ]:
s_cleaned_array_list = np.delete(s_cleaned_array_list,dl_list_ml,axis = 0)

labels = np.delete(labels,dl_list_ml,axis = 0)

In [ ]:
len(s_cleaned_array_list)

In [ ]:
labels_2c = np.delete(labels_2c,dl_list_ml,axis = 0)

In [ ]:
len(labels_2c)

In [ ]:
dl_list

In [ ]:
test_s = np.load('testdata_before_ml.npy',allow_pickle=True)

In [ ]:
dl_l = []

for i in range(len(s_cleaned_array_list)):
    
    for j in range(len(test_s)):
        
        if np.array_equal(test_s[j],s_cleaned_array_list[i]):
            
            dl_l.append(i)
            
            break
            
        else:
            
            continue

In [ ]:
len(dl_l)

In [ ]:
s_cleaned_array_list = np.delete(s_cleaned_array_list,dl_l,axis = 0)

labels = np.delete(labels,dl_l,axis = 0)

In [ ]:
len(s_cleaned_array_list)

In [ ]:
len(labels_2c)

In [ ]:
labels_2c = np.delete(labels_2c,dl_l,axis = 0)

In [ ]:
len(labels_2c)

In [ ]:
np.save('traindata.npy',s_cleaned_array_list)
np.save('trainlabels.npy',labels_2c)

In [ ]:
test_s[j] == s_cleaned_array_list[i]

In [ ]:
i

In [ ]:
dl_l

In [ ]:
test_s[-1] in s_cleaned_array_list

In [ ]:
test_s[85]

In [ ]:
np.isinf(test_s[-1],s_cleaned_array_list,)

In [ ]:
(test_s[-1] == test_s[-1]).all()

In [ ]:
mean_s_e = []

mean_s_p = []

mean_a_e = []

mean_a_p = []

std_s_e = []

std_s_p = []

std_a_e = []

std_a_p = []

for i in range(len(s_cleaned_array_list)):
      
    e_s = s_cleaned_array_list[i][:,:24:2]
    p_s = s_cleaned_array_list[i][:,1:24:2]
    
    e_a = a_cleaned_array_list[i][:,:24:2]
    p_a = a_cleaned_array_list[i][:,1:24:2]
        
    
    a = np.mean(e_s)
    b = np.mean(e_a)
    c = np.mean(p_s)
    d = np.mean(p_a)
       
    mean_s_e.append(a)
    mean_s_p.append(c)
    mean_a_e.append(b)
    mean_a_p.append(d)
    
    a = np.std(e_s)
    b = np.std(e_a)
    c = np.std(p_s)
    d = np.std(p_a)

    
    std_s_e.append(a)
    std_s_p.append(c)
    std_a_e.append(b)
    std_a_p.append(d)

In [ ]:
X_old = np.transpose(np.vstack((ett_slopes,ptt_slopes,mean_s_e,mean_s_p,mean_a_e,mean_a_p,std_s_e,std_s_p,std_a_e,std_a_p)))

In [ ]:
X = X_old
y = labels_2c

In [ ]:
X.shape

In [ ]:
for i in range(len(s_cleaned_array_list)):
    
    x_train, x_test, y_train, y_test = (np.vstack((X[:i],X[i+1:])) , X[i,:] , np.vstack((np.array(y[:i]).reshape((-1,1)),np.array(y[i+1:]).reshape((-1,1)))), y[i] )
    
    
    RF_clf =  SVC(C = 2  , class_weight = 'balanced', tol = 1e-6 , probability = True )


    RF_clf.fit(x_train,y_train)

    y_pred=RF_clf.predict(x_test.reshape(1, -1))
    
    prob = RF_clf.predict_proba(x_test.reshape(1, -1))[0]
    
    if y_test != y_pred[0]:
        
        print(i,y_test,prob,y_pred)

In [ ]:
for i in range(len(s_cleaned_array_list)):

    
    f = plt.figure(figsize=(15,5))
    ax = f.add_subplot(121)
    ax2 = f.add_subplot(122)
    plt.title(str(i))
    ax.plot(s_cleaned_array_list[i][:,:24:2].reshape(-1))
    ax2.plot(a_cleaned_array_list[i][:,:24:2].reshape(-1))
    
    plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 0.25 , random_state=128)

In [ ]:
train_scores = []
test_scores = []

for i in range(1,200):

    RF_clf =  SVC(C = i*0.01 + 0.01  , class_weight = 'balanced', tol = 1e-6 , probability = True )

    test_list = []
    train_list = []

    for j in range(1):

        RF_clf.fit(x_train,y_train)

        y_pred=RF_clf.predict(x_test)

        ts_RF = RF_clf.score(x_test,y_test)
        tr_RF = RF_clf.score(x_train,y_train)

        test_list.append(ts_RF)
        train_list.append(tr_RF)

    test_scores.append(np.mean(test_list))
    train_scores.append(np.mean(train_list))

In [ ]:
values = [i for i in range(1, 200)]
# plot of train and test scores vs tree depth
plt.plot(values, train_scores, '-o', label='Train')
plt.plot(values, test_scores, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 23)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

In [ ]:
bx_list = []

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 73)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 123)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 173)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 223)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 248)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 261)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 267)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
auc_scores_list = []

for i in range(1000):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size= 271)
    
    GSVM_clf = SVC(C =  1 , class_weight = 'balanced', tol = 1e-6 , probability = True , gamma = 'auto')
    GSVM_clf.fit(x_train,y_train)
    
    yproba = GSVM_clf.predict_proba(x_test)[::,1]
    auc_scores_list.append(roc_auc_score(y_test, yproba))
    
print(np.mean(auc_scores_list),np.std(auc_scores_list),np.max(auc_scores_list),np.min(auc_scores_list))

bx_list.append(auc_scores_list)

In [ ]:
plt.boxplot(bx_list)
plt.show()